# Methodology: Travel Time Matrices and Spatial Access Metrics  

Authors: [Irene Farah](https://www.linkedin.com/in/imfarah/),  [Julia Koschinsky](https://www.linkedin.com/in/julia-koschinsky-657599b1/), [Logan Noel](https://www.linkedin.com/in/lmnoel/).  
Contact: [Julia Koschinsky](mailto:jkoschinsky@uchicago.edu)  

Research assistance of [Shiv Agrawal](http://simonlab.uchicago.edu/people/ShivAgrawal.html), [Caitlyn Tien](https://www.linkedin.com/in/caitlyn-tien-0b784b161/) and [Richard Lu](https://www.linkedin.com/in/richard-lu-576874155/) is gratefully acknowledged.

Center for Spatial Data Science  
University of Chicago  

July 30, 2019

---
## Purpose and Notebook Structure

Across disciplines, spatial accessibility indicators allow you to address many questions, like who does and does not live within reach of specific amenities/services or where they might be spatial mismatches between supply and demand of these services. 

The purpose of this notebook is to present the methodology for 1) efficiently and transparently estimating network-based travel times or distances at scale (p2p module), and, based on this, 2) for generating the spatial access and coverage metrics, especially the access score (Model module). 

This notebook explains how the Python modules p2p and Models work that are part of the [spatial access package](https://pypi.org/project/spatial-access/). The p2p module is part of an open-source backend infrastructure for estimating network-based travel times for three travel modes: walking, driving, and biking. You can also read in a travel time matrix generated in OpenTrip Planner (otp). These **travel time matrices** serve as the input for the **access** and **coverage** **metrics** in Models.py to identify potential spatial access gaps. Access metrics are attributes of points of **origins** while coverage metrics are attributes of the **destination** points. 

The next notebook ([3_Travel_Time_Matrix.ipynb](./3_Travel_Time_Matrix.ipynb)) walks you through the computation of the travel time matrix, followed by three notebooks with demos of the metrics on spatial access ([4_Access_Metrics.ipynb](./4_Access_Metrics.ipynb)), coverage ([5_Coverage_Metrics.ipynb](./5_Coverage_Metrics.ipynb)), and two-stage floating catchment areas ([6_TSFCA.ipynb](./6_TSFCA.ipynb)).

As an example, we analyze health facilities in the City of Chicago with [public data](http://makosak.github.io/chihealthaccess/index.html). You are also encouraged to use your own data. We highlight the **_parameters_** you can specify as options for your own data.


---

## Motivation

Why did we decide to create a new package for computing travel times at scale?  

Compared to alternative state-of-the-art options, this package computes access **more efficiently** in an **open-source** and **scalable** framework that runs **offline** for confidential data.  

Generating large shortest path matrices for different travel modes is an important tool for spatial data science, but does not currently have a solution in Python that is open source, highly scalable and efficient. Several tools currently exist for similar purposes as this software package. OSRM, Valhalla, and OpenTripPlanner, among other services, offer matrix APIs to compute the shortest path distance for datasets but the open-source solutions break down when applied to very large datasets without dockerized solutions (same for pgrouting). On the other hand, both Graphhopper and GoogleMaps charge for the service, which becomes prohibitely expensive at scale. 

Each of the above services caps the number of entries in a request at 25-50, meaning that generating a matrix with 500,000 rows requires breaking the original matrix into millions of submatrices and making millions of individual queries. This approach works well for small datasets, but includes substantial overhead which is prohibitive on a large scale. The point-to-point shortest path algorithm presented here (p2p) can generate matrices between a set of origin and destination points (or origins-origins) in 2 lines of code, efficiently and with a low memory footprint. 

The example in this notebook generates a driving shortest path matrix for 46,251 blocks in Chicago in ~14 minutes (18 minutes for walking) whereas the same task took > 18 hours using Valhalla. For this particular dataset, the mean difference between time values for the driving shortest path matrix and Google Maps' Matrix API is 2 minutes.

---

## Overview of Travel Time Matrices 

Travel time matrices can be computed for walking, biking and driving times between origin and destination points. Instead, you can also choose to compute distances (in meters) between these points. We will refer to travel times by default in these notebooks (since this is the default setting and often of greater interest) but distances are implied and can easily be computed by changing one parameter (use_meters=True, as shown in the [TRAVEL TIME MATRIX DEMO](./3_Travel_Time_Matrix.ipynb)). 

There are two routes to compute these matrices: Creating **asymmetric** (nxm) or **symmetric** (nxn) matrices. Symmetric matrices are estimated origin to origin, while asymmetric matrices calculate origin to destination. You can generate a symmetric distance matrix and snap the points of interest to the matrix or create an asymmetric distance matrix that already incorporates origin and destination points. The symmetric approach is more appropriate when you need to calculate several metrics for the same area and different destinations.

---

## Overview of Spatial Access Metrics

After obtaining the travel times from origins to destinations (in this case, from the centroids of tracts to the health facilities), you can then calculate:

- **Access metrics (origin-based)**:
    - **Access Time**: Time to closest destinations (time to nearest neighbor)
    - **Access Count**: Count of destinations within a catchment area (e.g. how many destinations within a 30-minute walk?)
    - **Access Sum**: Sum of an attribute of destinations within a catchment area
    - **Access Model**: Score from each origin to destination (e.g. tract to health facilities) (gravity-model)
    - **Dest Sum**: Sum of destinations' attributes within areas (container approach)  
    
    
- **Coverage metrics (destination-based)**:  
    - **Coverage**: Supply-demand ratio for the extent of an area that a provider covers (e.g. for each hospital: number of physicians per patient within catchment area of hospital, also called physician-to-patient ratio)  


-  **Two Stage Floating Catchment Area (origin-based)**: Sum of coverage each point of origin has access to.  
    For details on the TSFCA click [here](https://journals.sagepub.com/doi/10.1068/b29120). 


## Package Structure and Workflow

This diagram shows how the package is structured: 

<img src="./figures/diagram_code.png" width="1400" title="Optional title">

The workflow first estimates a point-to-point shortest path [**(p2p)**](./spatial_access/p2p.py) algorithm for creating the travel-time matrix by travel mode (walking/driving/biking). The code takes the outermost values of the origins and creates a bounding box using their latitude and longitude (destinations need to be constrained to the spatial extent of the origins). Once it generates the bounding box, it queries the network data from OSM, retrieving information on different types of roads and building a graph. Based on this, p2p then creates the travel time matrix.  

Then, it creates a base model infrastructure [**(BaseModel)**](./spatial_access/BaseModel.py) for creating the metrics, using the BaseModel class (parent of Models.py). Specifically, this class allows the user to generate any type of metric, suiting each user's needs. Finally, it creates the models [**(Models)**](./spatial_access/Models.py) for creating aggregate measures of the Access Model, AccessTime, AccessCount, AccessSum, DestinationSum, Coverage Score, and TSFCA. 

This framework provides the user with the flexibility to start at different stages along the process:  

1) Start by creating an asymmetric travel time matrix using the p2p algorithm.  
2) Start by creating a symmetric travel time matrix using the p2p algorithm and then subsetting it to create an asymmetric travel time matrix.  
3) Input an external travel time matrix and run the metrics.  

---
## How Travel Time Matrices Work
_(Disregard this section if you already have a travel-time matrix.)_

**_Input Requirements_**  

In order to construct the travel time matrices, the csv table should contain **ID, latitude, longitude** variables for the origins and destinations.  
Destinations need to be constrained to the spatial extent of the origins.  

**OpenStreetMap (OSM) structure**  

To better understand how the algorithm computes travel times, a brief description of OSM's structure follows. OSM's data structure is composed of four elements: nodes, ways, relations, and tags.
Nodes are latitude and longitude coordinates (projected in WGS 84) that represent the map’s features. Ways are a list of nodes that compose the geometry features (i.e. point, line, polygon) within a map, depicting streets, waterways, parks, etc. Relations express the relationship between nodes and ways. Lastly, tags are attached to nodes, ways or relations, storing metadata about the map objects.  

We download the OpenStreetMap network using the area of the previously determined bounding box (i.e. the area of interest defined by the latitude and longitude coordinates). The complexity of the network depends on the number of nodes within this bounding box. In contrast, the number of observations should not affect the efficiency of the running times. In order to get the distances from OSM, OSM-Net calculates the distances of the relations, creating the edges that are queried for the travel time estimation. To estimate these distances, both origin and destination files should be using the same WGS 84 coordinate reference (EPSG:4326).

**P2P (point to point) algorithm**  
In order to calculate the network distance matrix, first, the code extracts the outermost value of latitude/longitude from the origin input table to create a bounding box of the area of interest. The size of the bounding box is buffered, specifically it is increased by 'epsilon', to avoid cutting off the network of datapoints near the boundary of the bounding box. The user can tweak the value of epsilon in **Configs.py**.

P2P uses a k-d tree to match each point in the origin and destination data to its nearest neighbor node in the OSM network, and then finds the Vincenty distance between the two points. Vincenty's formulae estimate the geodesic distance between two points according to an ellipsoidal model of the Earth. 

For the travel time computation between origin and destination, the classic Dijkstra's algorithm is then applied to consider every possible route and then select the fastest route. Therefore, P2P also uses an adjacency list representation for Dijkstra's algorithm to find the shortest path for every node to every other node in the underlying OSM network, but it can skip doing any processing for nodes that do not have an attached origin data point. The advantage of this approach is that it scales to very large datasets; as opposed to the adjacency matrix representation (which can easily exceed the memory of many systems for reasonably large datasets). P2P never loads the entire network into memory at one time, meaning the memory footprint is relatively small. This also means the multi-threaded performance of P2P greatly outperforms the single-threaded performance.  

For every point in the origin dataset to every point in the destination dataset, the base impedence is the cost found using Dijkstra. To the base value we add the 'last mile' inferred impedence from the origin and destination points to their respective nearest nodes, determined by the Euclidean distance and a constant traversal speed. The 'last mile' is figurative; in the City of Chicago, for instance, 75 percent of block centroids were within 100 meters of the nearest OSM node and 95 percent of block centroids were within 200 meters.  

**Islands**  
Some of the units of analysis are classified as islands (disconnected nodes) by OSM. Therefore, Kosaraju's algorithm for directed graph strong connectedness is implemented in p2p (lines 713 - 805 of p2p.py under \_request_network2 function). In graph theory, strong connectivity means that a path exists between any pair of nodes. Thus, we implement Kosaraju's algorithm to identify the disconnected nodes and we delete them from the network. 

**Script**  
The p2p.py script runs the point to point (p2p) algorithm and creates the class **TransitMatrix**. The output of p2p is the travel time matrix, which is computed in seconds. The **TransitMatrix** unified class run manages all aspects of computing a transit time matrix where matrices can be symmetric or asymmetric (as mentioned above). Therefore, load one input file if you want a symmetric distance graph, or two for an asymmetric matrix. Particularly, this class accounts for all the details that entail specifying the speed limits, creating the bounding box for the area of interest in order to run the OSM query, and calculating the shortest path matrix.  

**Specifics of P2P parameters**  
Several parameters should be taken into account when calculating the distance network matrix:  
* The **network type** can be determined for walking, biking, or driving.  


* Thresholds can be adjusted and are considered in the calculation of the distance matrix: the **average walking speed** is 5 km/h (3 mph) and the default **average driving speed** is 40 km/h (25 mph).  You can adjust this parameter for different populations.  For example, [Chicago](https://www.cityofchicago.org/dam/city/depts/cdot/StreetandSitePlanDesignStandards407.pdf) estimates an average block dimension of 660 feet (200 m) by 330 feet (100 m). These dimensions might change across cities; therefore, the average walking speed of 3 mph estimates that a person, on average, walks a block in 72 to 144 seconds (1.2 - 2.4 min). The default average speeds and speed limits for different OSM type of roads can be found in **Config.py** and specified when running the matrices.   
  
  
* Also for walking and driving, you can specify a **node penalty** of X seconds for the number of intersections within the area of analysis. The logic is that having more intersections will increase the travel time due to crossings. However, by doing a time travel calibration between the p2p algorithm and GoogleMaps, there was no need for adding penalities for the city of Chicago for walking and biking, but we added 4 seconds for driving. It can be specified within the **Configs.py** file.
  
  
* For driving, the network is **directed**, meaning that one-way streets are respected and A->B and B->A can have different edge traversal speeds.  


* **Epsilon**: Controls how large to make the network bounding box beyond your dataset. Larger epsilons result in longer computation times, but smaller epsilons result in slightly reduced accuracy at the very edges of the bounding box, especially for driving networks. The default is currently set at 0.05, which seems to balance the two reasonably well. (+/-) 0.02 will result in a large increase/decrease in computation time and accuracy. If too many values are defined as -1, it means that the epsilon is too small. Refer to the epsilon calibration to assess if this value must change and the matrix contains too many -1. The value of -1 is hardcoded in the tmat.h file and is considered as an NaN value of the origins when estimating the metrics.  


* The package allows output of travel time matrices either in **seconds** or in **meters**. The user can specify the output in meters when running the matrix using `use_meters=True`.


GO TO [TRAVEL TIME MATRIX DEMO](./3_Travel_Time_Matrix.ipynb) 


<a id='communityanalytics'></a>

---
## Specification of Spatial Access Metrics
_(Disregard this section if you only care about the travel-time matrix.)_

## Origin-based Metrics

The metrics covered in this section are attributes of the origin points, i.e. they considere spatial access from the perspective of someone accessing amenities. In contrast, the metrics in the next section are attributes of the destination, i.e. they consider spatial access from the perspective of the service provider. In addition, the 2-stage floating catchment area model is an origin-based metric that combined spatial access and coverage elements.

This spatial access package allows you to compute the following metrics that are attributes of the point of origin:

**1. Access Time**

Shortest time to the nearest facility/amenity.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **_Input Requirements:_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; csv file: **ID, latitude, longitude** for origins and destinations  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **category**  for sub-setting  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **larger areal ID**  for aggregating.

**2. Access Count**

Total number of amenities/facilities within the catchment area 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **_Input Requirements:_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; csv file: **ID, latitude, longitude** for origins and destinations  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **category**  for sub-setting  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **larger area ID**  for aggregating.

**3. Access Sum**

Captures the sum of an attribute within a catchment area.  
(e.g. number of doctors within a 30-minute walk from the origins)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **_Input Requirements:_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; csv file: **ID, latitude, longitude** for origins and destinations 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **capacity** for destinations  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **category**  for sub-setting  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **larger area ID**  for aggregating.

**4. Dest Sum (container approach)**

Captures the sum of the attributes of a destination, within an area.  
(e.g. number of doctors within a community area - does not require travel time matrix)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **_Input Requirements:_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; csv file: **ID, latitude, longitude** for origins and destinations 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **capacity** for destinations  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **category**  for sub-setting  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **larger area ID**  for aggregating. 

**5. Access Model**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **_Input Requirements:_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; csv file: **ID, latitude, longitude** for origins and destinations  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **category**  for sub-setting  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **larger areal ID**  for aggregating.

---
**How the Access Score Works**

The Access Model generates an access score to measure how accessible a location is to multiple amenities within a given travel time (e.g. 20 minutes walking). In our example, tract centroids are points of origin and health facilities are destination points. 

The score is a weighted sum. Every destination point receives a value that represents the product of the following weights, which are then summed across destinations within a travel time of the point of origin to obtain the final score: 

1) **distance decay** where closer amenities have more weight (default = linear)  
2) **relative importance of an amenity type** (e.g. with a greater weight for supermarkets than museums)  
3) **variety / penalty for same types** (where more of the same type of amenity has less weight). 

This section explains how the three weights work and how the score is then constructed.

---

**Distance Decay**

Distance decay weights are applied to give closer amenities more weight and reduce the weight of more distant ones. Amenities beyond the specified travel time threshold (e.g. 30 min walk) are not considered in the score. 

In more technical terms, the distance decay function describes the decreasing intensity of a value as the distance increases. You can add any function in the code, depending on your amenities' intensity behavior. Out of the box, this package provides the three functions shown below: linear, square root, and logit (default = linear):

<img src="./figures/dd.png" width="1400" title="Optional title">

---

**Relative Importance and Variety**

You can create the access score for one type of amenity (e.g. supermarkets) or a variety of types (e.g. supermarkets, museums and restaurants). In both cases, you have the option to manually assign the relative importance of amenities and give less weight to the same type of amenities. For instance, you can up-weight larger supermarkets or supermarkets vs restaurants vs. convenience stores and downweight any additional restaurant beyond the first few within a travel time. If you have a variety of types, you can compute the score for the pooled categories (supermarkets, museums and restaurants together) or for each category separately.

You can estimate the score with normalization (0-100) or without (and then compare intervals like quintiles across place or time).   

The dictionary below shows an example of the weights assigned to each amenity: 


In [ ]:
#Example of importance and variety weights:

dict = {
"Hospitals": [10,10,10,10,10],
"Federally Qualified Health Centers": [8, 7, 6, 5, 4],
"School-Based Health Centers": [7, 7, 6, 6, 5],
"All Free Health Clinics": [5, 5, 5, 4, 4],
"Other Health Providers": [4,3,2,1,1]
}

#Make sure your categories in the dictionary match the spelling in the csv files - otherwise they will be ignored.


You can specify the weights based on your research needs. In this case, a hospital will be categorized as more important than a smaller free health clinic (10 vs 5 for the first of each facility). Moreover, the dictionary categorizes the second nearest FQHC as having less weight than the first one (8 vs 7). However, additional hospitals are not down-weighted since the demand for hospitals usually exceeds supply. In other words, the 5th hospital has the same weight as the 1st. If there is a sixth hospital within 30 minutes of a tract center, the score will neglect it since there are only 5 weights specified under 'Hospitals', so you want to make sure that your weight count equals or exceeds your destination count within the travel time of your point of origin. As mentioned before, destinations beyond the travel time threshold are ignored. 


**How the Access Score is Calculated**

The figure below shows a point of origin (black x) and the three health facilities that can be reached within a 30 minute walk from there: two FQHCs (blue dots, 12 and 25 minutes away) and one hospital (red dot, 10 minutes away). The table next to the map lists these three facilities, the travel times to them, and their respective importance and variety weights, specified in the weights dictionary above. The closest facility, the hospital, is weighted as 10, followed by the next closest Federally Qualified Health Center, weighted as 8 and the third closest health center, weighted as 7,

The last column contains the  weights from the distance decay function shown in the image to the right of the table. The distance decay function weighs each destination depending on its relative distance to the point of origin: closer destinations are weighed higher that more distant destinations. 

This image shows how the distance decay function maps a given travel time (x-axis) to a score from 0-1 (y-axis). The 10-minute travel time to the hospital is weighted by the distance decay function with a score of 0.9 (and smaller weights of 0.81 and 0.1 for the other two facilities at the larger 12 and 25 min distances). 

The score for each facility Is the product of the importance/variety weights and these distance decay weights: As shown in the table below, these scores are 9, 6.48 and 0.70 for the hospital and two FQHCs, respectively. The final score is the sum of these facility scores: In this case, it is 16.18.

Note that the more categories you have, the larger your score will be. By default, the score is not normalized to observe the overall distribution across places and time, but the results can also be standardized. 


<img src="figures/flow.png" width="1000" title="Optional title"  align="center">

**_Specifications_**  

In the **demo prompt**, you can specify parameters with two different commands:  

**name.AccessModel( ):**  
- network_type ('walk', 'bike', 'drive', 'otp') 
- sources_filename (primary input data)
- destinations_filename (secondary input data)
- source_column_names (dictionary that maps column names to expected values)
- dest_column_names (dictionary that maps column names to expected values)
- transit_matrix_filename (sources-destination travel time matrix)
- decay_function ('linear', 'root', 'logit', default is 'linear')  

**name.calculate():**
- upper_threshold (travel time threshold in seconds, default is 30 minutes; beyond the threshold, score will be zero)
- category_weight_dict (specifies the weights of each destination defined as dictionary, default dictionary will contain [1,1,1,1,1,1,1,1,1,1] weights.)
- normalize (accepts boolean, default is False and shows only non-normalized results, true shows normalized values.)
- normalize_type ('z_score' or 'minmax', default is 'minmax')

GO TO [ACCESS SCORE DEMO](./4_Access_Metrics.ipynb) 


----  

## Destination-based Metrics  

**1. Coverage**

The **Coverage** model generates a coverage access which shows the per capita spending available to a specific targeted population. The model focuses on the coverage of the destination, scrutinizing how many people are within a catchment area. Specifically, it takes the total spending of the facility/establishment and divides it by the total population it serves within a buffer (in this case, 30 minutes). In the specifications, the magnitude of the destination is denominated as target. 

 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **_Input Requirements:_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; csv file: **ID, latitude, longitude** for origins and destinations  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **population** (origins)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **capacity**  (destinations)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **category**  for sub-setting (destinations)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **larger area ID**  for aggregating.

<img src="figures/coverage.png" width="600" title="Optional title"  align="center">

 

**_Specifications_**  

**name.Coverage( ):**  
- network_type ('walk', 'bike', 'drive', or 'otp')
- sources_filename (primary input data)
- destinations_filename (secondary input data)
- transit_matrix_filename (origin-destination transit matrix)
        
**name.calculate():**
- upper_threshold (travel time threshold in seconds; beyond the threshold, score will be zero)


GO TO [COVERAGE SCORE DEMO](./5_Coverage_Metrics.ipynb) 

----
## Two-Stage Floating Catchment Area Model  

The **TSFCA** model generates a coverage access which shows the per capita spending available to a specific targeted population. The model focuses on the coverage of the destination, scrutinizing how many people are within a catchment area. Specifically, it takes the total spending of the facility/establishment and divides it by the total population it serves within a buffer (in this case, 30 minutes). In the specifications, the magnitude of the destination is denominated as target. 



<img src="figures/tsfca.png" width="1100" title="Optional title"  align="center">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **_Input Requirements:_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; csv file: **ID, latitude, longitude** for origins and destinations  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **population** (origins)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **capacity**  (destinations)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **category**  for sub-setting (destinations)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `+` **larger area ID**  for aggregating.

GO TO [TSFCA SCORE DEMO](./6_TSFCA.ipynb) 

---

## Subsetting, Aggregation and Plotting


The metrics can also be **subset** by categories. If you have many types of amenities, you can choose 1 to n categories to calculate the metrics for each category (e.g., health clinics and hospitals as opposed to all health facilities).

The scores can also be **aggregated** at a larger areal unit to show overall access patterns, as is shown in the demos. 

The scripts also contain hard-coded empirical cumulative distribution function and choropleth **plots** to preview data patterns (see the notebook demos). However, these plots are not designed for presentation purposes. To create professional graphs and maps, the results saved in the csv files can be merged to the origin's or destination's shapefile for mapping and plotting in other software.  